In [1]:
import pandas as pd
import numpy as np
import os
import glob
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
import wfdb
import statsmodels.api as sm
import matplotlib.image as mpimg
import tkinter as tk
from matplotlib.figure import Figure
from IPython.display import display, HTML
import csv
from collections import defaultdict
from sklearn.metrics import r2_score
import matplotlib.ticker as ticker
import pickle
from sklearn.linear_model import LogisticRegression
import tensorflow.keras as ks
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import backend as Ks
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from numpy.random import seed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [2]:
def getCNNModeltoInterpret(fname):
    learning_rate = 0.1
    momentum = 0.0
    model = ks.Sequential()
    # Conv Layer 1
    model.add(ks.layers.Conv1D(filters=10, kernel_size=5, strides=1, input_shape=(None,3),
                           padding='same',use_bias=True, 
                           kernel_initializer='glorot_uniform', bias_initializer='zeros'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    # Conv Layer 2-11 (10 layers)
    for layernum in range(10):
        model.add(ks.layers.Conv1D(filters=10, kernel_size=5, strides=1,
                           padding='same',use_bias=True, 
                           kernel_initializer='glorot_uniform', bias_initializer='zeros'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
    
    # Conv Layer 12
    model.add(ks.layers.Conv1D(filters=1, kernel_size=5, strides=1,
                           padding='same',use_bias=True, 
                           kernel_initializer='glorot_uniform', bias_initializer='zeros'))
    model.add(Activation('sigmoid'))
    previousModel = ks.models.load_model(fname)
    model.set_weights(previousModel.get_weights())
    return model

def getCNNSegMap(model, dfstat, data):
    TOTAL_LEN = np.shape(data)[0]

    mRR = dfstat['Mean'][dfstat['Parameter']=='RR'].values[0]
    sdRR = dfstat['StdDev'][dfstat['Parameter']=='RR'].values[0]
    mSpO2 = dfstat['Mean'][dfstat['Parameter']=='SpO2'].values[0]
    sdSpO2 = dfstat['StdDev'][dfstat['Parameter']=='SpO2'].values[0]
    mHR = dfstat['Mean'][dfstat['Parameter']=='HR'].values[0]
    sdHR = dfstat['StdDev'][dfstat['Parameter']=='HR'].values[0]

    rrnaf = (data['RRraw'].values - mRR)/sdRR
    spo2naf = (data['SpO2raw'].values - mSpO2)/sdSpO2
    hrnaf = (data['HRraw'].values - mHR)/sdHR
    
    X_TEST = np.zeros((1,TOTAL_LEN,3), dtype=np.float32)
    X_TEST[0,:,0] = rrnaf
    X_TEST[0,:,1] = spo2naf
    X_TEST[0,:,2] = hrnaf
    predictions = model.predict(X_TEST)
    return predictions[0,:,0]

def getPosProbSAX(refProbDf, dfstat, data):
    mRR = dfstat['Mean'][dfstat['Parameter']=='RR'].values[0]
    sdRR = dfstat['StdDev'][dfstat['Parameter']=='RR'].values[0]
    mSpO2 = dfstat['Mean'][dfstat['Parameter']=='SpO2'].values[0]
    sdSpO2 = dfstat['StdDev'][dfstat['Parameter']=='SpO2'].values[0]
    mHR = dfstat['Mean'][dfstat['Parameter']=='HR'].values[0]
    sdHR = dfstat['StdDev'][dfstat['Parameter']=='HR'].values[0]
    tdf = pd.DataFrame(columns=['RR','SpO2','HR','nRR','nSpO2','nHR','symRR','symSpO2','symHR','Symbol','Label'])
    tdf['RR'] = data['RRraw'].values
    tdf['SpO2'] = data['SpO2raw'].values
    tdf['HR'] = data['HRraw'].values
    tdf['nRR'] = (data['RRraw'].values - mRR)/sdRR
    tdf['nSpO2'] = (data['SpO2raw'].values - mSpO2)/sdSpO2
    tdf['nHR'] = (data['HRraw'].values - mHR)/sdHR
    tdf['Label'] = data['Label'].values
    cutpoints = [-np.inf,-1.5,-1,-0.5,0,0.5,1,1.5,np.inf]
    tdf['symRR'] = pd.cut(tdf['nRR'], bins = cutpoints, labels=["A","B","C","D","E","F","G","H"])
    tdf['symSpO2'] = pd.cut(tdf['nSpO2'], bins = cutpoints, labels=["A","B","C","D","E","F","G","H"])
    tdf['symHR'] = pd.cut(tdf['nHR'], bins = cutpoints, labels=["A","B","C","D","E","F","G","H"])
    tdf['Symbol'] = tdf.apply(lambda row: row['symRR']+ row['symSpO2']+row['symHR'],axis=1)
    newtdf = pd.merge(tdf , refProbDf[['Symbol', 'PosProb']], how = 'left', left_on='Symbol', right_on='Symbol')
    return newtdf['PosProb'].values

def getSAXfromNormSeries(refProbDf,nRR,nSpO2,nHR):
    tdf = pd.DataFrame(columns=['nRR','nSpO2','nHR','symRR','symSpO2','symHR','Symbol'])
    tdf['nRR'] = nRR
    tdf['nSpO2'] = nSpO2
    tdf['nHR'] = nHR
    cutpoints = [-np.inf,-1.5,-1,-0.5,0,0.5,1,1.5,np.inf]
    tdf['symRR'] = pd.cut(tdf['nRR'], bins = cutpoints, labels=["A","B","C","D","E","F","G","H"])
    tdf['symSpO2'] = pd.cut(tdf['nSpO2'], bins = cutpoints, labels=["A","B","C","D","E","F","G","H"])
    tdf['symHR'] = pd.cut(tdf['nHR'], bins = cutpoints, labels=["A","B","C","D","E","F","G","H"])
    tdf['Symbol'] = tdf.apply(lambda row: row['symRR']+ row['symSpO2']+row['symHR'],axis=1)
    newtdf = pd.merge(tdf , refProbDf[['Symbol', 'PosProb']], how = 'left', left_on='Symbol', right_on='Symbol')
    return newtdf['PosProb'].values

def movingAvg(x,n):
    y = np.zeros(len(x),dtype=np.float32)
    ysum = 0
    for i in range(len(x)):
        if(i<n):
            ysum += x[i]
            y[i] = ysum/float(i+1)
        else:
            ysum = ysum + x[i] - x[i-n]
            y[i] = ysum/float(n)
    return y

def getLSTMModeltoInterpret(fname):
    previousModel = ks.models.load_model(fname)
    model = Sequential()
    model.add(LSTM(100, input_shape=(None, 3),return_sequences=True))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    model.set_weights(previousModel.get_weights())
    return model

def getLSTMSegMap(model, dfstat, data):
    TOTAL_LEN = np.shape(data)[0]

    mRR = dfstat['Mean'][dfstat['Parameter']=='RR'].values[0]
    sdRR = dfstat['StdDev'][dfstat['Parameter']=='RR'].values[0]
    mSpO2 = dfstat['Mean'][dfstat['Parameter']=='SpO2'].values[0]
    sdSpO2 = dfstat['StdDev'][dfstat['Parameter']=='SpO2'].values[0]
    mHR = dfstat['Mean'][dfstat['Parameter']=='HR'].values[0]
    sdHR = dfstat['StdDev'][dfstat['Parameter']=='HR'].values[0]

    rrnaf = (data['RRraw'].values - mRR)/sdRR
    spo2naf = (data['SpO2raw'].values - mSpO2)/sdSpO2
    hrnaf = (data['HRraw'].values - mHR)/sdHR
    
    X_TEST = np.zeros((1,TOTAL_LEN,3), dtype=np.float32)
    X_TEST[0,:,0] = rrnaf
    X_TEST[0,:,1] = spo2naf
    X_TEST[0,:,2] = hrnaf
    predictions = model.predict(X_TEST)
    return predictions[0,:,0]

In [ ]:
# Read p file corresponding to serial id as this will contain label of series
# Read records using wfdb
# Match lengths
serial_id = 141
refProbDf = pd.read_csv('SAXProb.csv')
annofile = r'RDSegAnno.csv'
model = getCNNModeltoInterpret('SegCNN.h5')
LSTMmodel = getLSTMModeltoInterpret(fname)

dfOuranno = pd.read_csv(annofile, encoding='iso-8859-1')  
record_name = dfOuranno.loc[serial_id,'RecordNum'] 
flname = './FinalRecordsSeg/'+str(record_name)+'n.hea'
recname = './FinalRecordsSeg/'+str(record_name)+'n'       
[samples,R,S,H,firstline] = getIndexOfRRSpO2HR(flname)
rec =  wfdb.io.rdrecord(str(recname))
xrrFull = rec.p_signal[:,R]
xspo2Full = rec.p_signal[:,S]
xhrFull = rec.p_signal[:,H]
TOTAL_LEN_FULL = len(xrrFull)
print(TOTAL_LEN_FULL)
tFull = np.arange(0,TOTAL_LEN_FULL,1)
tnewFull = tFull/60.0
myfn = str(serial_id)
myfn = myfn.zfill(3)
myfname = r'./SeriesSegData/p'+myfn+'.csv'
SegAnnoDf = pd.read_csv(myfname, encoding='iso-8859-1')
LabelsFull = SegAnnoDf['Label'].values
fstat = './'+'parStat'+'.csv'
StatDf = pd.read_csv(fstat, encoding='iso-8859-1')
RRMean = StatDf[StatDf['Parameter']=='RR']['Mean'].values[0]
RRStdDev = StatDf[StatDf['Parameter']=='RR']['StdDev'].values[0]
SpO2Mean = StatDf[StatDf['Parameter']=='SpO2']['Mean'].values[0]
SpO2StdDev = StatDf[StatDf['Parameter']=='SpO2']['StdDev'].values[0]
HRMean = StatDf[StatDf['Parameter']=='HR']['Mean'].values[0]
HRStdDev = StatDf[StatDf['Parameter']=='HR']['StdDev'].values[0]

OUTLIERSD = 2.5
LOWESSWINDOW = 3.0
#fig = plt.figure(figsize=(10.5,8))
STEP = 60
j = 0
for i in range(0,TOTAL_LEN_FULL,STEP):
    fig = plt.figure(figsize=(10.5,8))
    plt.clf()
    ax = plt.axes(xlim=(-5,max(tnewFull)+5), ylim=(-30,135))
    
    xrr = xrrFull[0:i+STEP]
    xspo2 = xspo2Full[0:i+STEP]
    xhr = xhrFull[0:i+STEP]
    t = tFull[0:i+STEP]
    tnew = tnewFull[0:i+STEP]
    Labels = LabelsFull[0:i+STEP]
    TOTAL_LEN = len(xrr)

    
    
    [rrnaf,t] = fillNAzero(xrr,t)
    [spo2naf,t] = fillNAzero(xspo2,t)
    [hrnaf,t] = fillNAzero(xhr,t)
    
    [xrrnew,t] = outlierRejector(xrr,t,OUTLIERSD)
    [xspo2new,t] = outlierRejector(xspo2,t,OUTLIERSD)
    [xhrnew,t] = outlierRejector(xhr,t,OUTLIERSD)

    zrrnew = applyLowess(xrrnew,t,LOWESSWINDOW*60)
    zspo2new = applyLowess(xspo2new,t,LOWESSWINDOW*60)
    zhrnew = applyLowess(xhrnew,t,LOWESSWINDOW*60)
    
    rrnafNorm = (rrnaf - RRMean)/RRStdDev
    spo2nafNorm = (spo2naf - SpO2Mean)/SpO2StdDev
    hrnafNorm = (hrnaf - HRMean)/HRStdDev

    X_TEST = np.zeros((1,TOTAL_LEN,3), dtype=np.float32)
    X_TEST[0,:,0] = rrnafNorm
    X_TEST[0,:,1] = spo2nafNorm
    X_TEST[0,:,2] = hrnafNorm
    
    plt.scatter(tnew,xrr,c='hotpink',marker='.',alpha='0.1')
    plt.scatter(tnew,xhr,c='magenta',marker='.',alpha='0.1')
    plt.scatter(tnew,xspo2,c='cornflowerblue',marker='.',alpha='0.05')
    
    plt.plot(zrrnew[:,0]/60.0,(zrrnew[:,1]),'--r',label='RR',linewidth=2)
    plt.plot(zhrnew[:,0]/60.0,(zhrnew[:,1]),'--m',label='HR',linewidth=2)
    plt.plot(zspo2new[:,0]/60.0,(zspo2new[:,1]),'--b',label='SpO2',linewidth=2)
    
    # Every time we can model.predict size of X_TEST is different
    predictions = model.predict(X_TEST)
    movingAvgPredictions = movingAvg(predictions[0,:,0],120)
    
    predictionsLSTM = 
    
    
    SI = round(movingAvgPredictions[-1]*100)
    SIstr = str(SI)+'%'
    SAXprob = getSAXfromNormSeries(refProbDf,rrnafNorm,spo2nafNorm,hrnafNorm)
    avgSAXprob = movingAvg(SAXprob,120)
    T = round(tnew[-1])
    Tstr = str(T)+' hrs'
    
    plt.plot(tnew,20*Labels-25.0,label='Clinical',linewidth=2)
    # Plot CNN results
    plt.plot(tnew,20*movingAvgPredictions-25.0,label='SI CNN',linewidth=2)
    # Plot SAX results
    #plt.plot(tnew,20*avgSAXprob-25.0,label='SAX',linewidth=2)
    plt.legend(loc=2)
        
    plt.text(0,51,Tstr, fontsize=15)
    plt.text(0,31,SIstr, fontsize=15)
    filename='./ForGIF/step'+str(j).zfill(3)+'.png'
    plt.savefig(filename, dpi=96)
    j = j+1
    plt.gca()
    #plt.show()